In [307]:
import pandas as pd
from utils import *
from mappings import *

In [308]:
df = read_dbf('../data/raw/exped.DBF')

In [309]:
df.head()

,expid,peakid,year,season,host,route1,route2,route3,route4,nation,...,accidents,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum
0,ANN260101,ANN2,1960,1,1,NW Ridge-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2442047
1,ANN269301,ANN2,1969,3,1,NW Ridge-W Ridge,None,None,None,Yugoslavia,...,Draslar frostbitten hands and feet,None,None,None,None,False,False,None,None,2445501
2,ANN273101,ANN2,1973,1,1,W Ridge-N Face,None,None,None,Japan,...,None,None,None,None,None,False,False,None,None,2446797
3,ANN278301,ANN2,1978,3,1,N Face-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2448822
4,ANN279301,ANN2,1979,3,1,N Face-W Ridge,NW Ridge of A-IV,None,None,UK,...,None,None,None,None,None,False,False,None,None,2449204


### Primary key

In [310]:
df.expid = df.expid.str.cat(df.year.astype(str), sep='_')

In [311]:
df.head()

,expid,peakid,year,season,host,route1,route2,route3,route4,nation,...,accidents,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum
0,ANN260101_1960,ANN2,1960,1,1,NW Ridge-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2442047
1,ANN269301_1969,ANN2,1969,3,1,NW Ridge-W Ridge,None,None,None,Yugoslavia,...,Draslar frostbitten hands and feet,None,None,None,None,False,False,None,None,2445501
2,ANN273101_1973,ANN2,1973,1,1,W Ridge-N Face,None,None,None,Japan,...,None,None,None,None,None,False,False,None,None,2446797
3,ANN278301_1978,ANN2,1978,3,1,N Face-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2448822
4,ANN279301_1979,ANN2,1979,3,1,N Face-W Ridge,NW Ridge of A-IV,None,None,UK,...,None,None,None,None,None,False,False,None,None,2449204


### Expedition Countries
#### Host

In [312]:
# replace documentation id mapping with custom mapping
df.host = apply_map(df.host, host_map)
df = update_country_list(df, 'host')

#### Nation

In [313]:
# explode slash-separated values into scalar names
df_nations = df[['expid', 'nation']]
df_nations.nation = df_nations.nation.str.split('/')
df_nations = df_nations.explode('nation', ignore_index=True)

In [314]:
# swap names with country ids
df_nations = update_country_list(df_nations, 'nation')

In [315]:
df.drop('nation', axis=1, inplace=True)

#### Countries

In [316]:
df.countries.head()

0    India, Nepal
1            None
2            None
3            None
4            None
Name: countries, dtype: object

In [317]:
# explode comma-separated country names into scalar values
df_countries = df[['expid', 'countries']].rename({'countries': 'country'}, axis=1)

df_countries.country = df_countries.country.str.split(',')
df_countries = df_countries.explode('country', ignore_index=True)

df_countries.country = df_countries.country.str.split('/')
df_countries = df_countries.explode('country', ignore_index=True)

In [318]:
df_countries.country.unique()

array(['India', ' Nepal', None, 'New Zealand', 'Nepal', 'USA', 'Austria',
       ' Italy', ' Switzerland', 'UK', ' France', ' W Germany',
       'Switzerland', 'France', 'Netherlands', 'Germany', ' Spain', ' UK',
       ' USA', 'Australia', 'Ireland', ' Netherlands', 'Czechoslovakia',
       'Canada', 'Belgium', ' Poland', 'Poland', 'China', 'Italy',
       'W Germany', 'Spain', ' Canada', ' Denmark', 'Brazil', 'Iceland',
       'Liechtenstein', 'Andorra', ' Ireland', 'Luxembourg', ' Mexico',
       'Argentina', ' India', 'Mexico', ' Belgium', ' New Zealand',
       ' Sweden', 'Sweden', 'Japan', 'Norway', 'S Korea', ' Germany',
       ' Portugal', ' Czechoslovakia', 'Bulgaria', 'Ukraine', 'Finland',
       ' Luxembourg', ' USSR', 'USSR', ' Norway', ' Hong Kong',
       'Indonesia', 'Ecuador', ' Yugoslavia', ' Kenya', 'Denmark',
       'Chile', 'S Africa', ' Israel', 'Kazakhstan', 'Croatia',
       ' Bulgaria', 'N Korea', ' Latvia', ' Peru', 'Portugal', 'Georgia',
       'Belarus', ' Sl

In [319]:
# swap names with country ids
df_countries = update_country_list(df_countries, 'country')

In [320]:
df.drop('countries', axis=1, inplace=True)

### Routes

In [321]:
route_1_df = df.loc[
	df.route1.notna() | df.success1.notna() | df.ascent1.notna(),
	['expid', 'route1', 'ascent1', 'success1']
]
route_1_df.rename({'route1': 'route', 'ascent1': 'ascent', 'success1': 'success'}, axis=1, inplace=True)
route_1_df['number'] = 1

In [322]:
route_2_df = df.loc[
	df.route2.notna() | df.success2.notna() | df.ascent2.notna(),
	['expid', 'route2', 'ascent2', 'success2']
]
route_2_df.rename({'route2': 'route', 'ascent2': 'ascent', 'success2': 'success'}, axis=1, inplace=True)
route_2_df['number'] = 2

In [323]:
route_3_df = df.loc[
	df.route3.notna() | df.success3.notna() | df.ascent3.notna(),
	['expid', 'route3', 'ascent3', 'success3']
]
route_3_df.rename({'route3': 'route', 'ascent3': 'ascent', 'success3': 'success'}, axis=1, inplace=True)
route_3_df['number'] = 3

In [324]:
route_4_df = df.loc[
	df.route4.notna() | df.success4.notna() | df.ascent4.notna(),
	['expid', 'route4', 'ascent4', 'success4']
]
route_4_df.rename({'route4': 'route', 'ascent4': 'ascent', 'success4': 'success'}, axis=1, inplace=True)
route_4_df['number'] = 4

In [325]:
df_routes = pd.concat([route_1_df, route_2_df, route_3_df, route_4_df], ignore_index=True)

In [326]:
df_routes.head()

,expid,route,ascent,success,number
0,ANN260101_1960,NW Ridge-W Ridge,1st,True,1
1,ANN269301_1969,NW Ridge-W Ridge,2nd,True,1
2,ANN273101_1973,W Ridge-N Face,3rd,True,1
3,ANN278301_1978,N Face-W Ridge,None,False,1
4,ANN279301_1979,N Face-W Ridge,None,False,1


In [327]:
# remove route columns
df.drop(
	['route1', 'ascent1', 'success1', 'route2', 'ascent2', 'success2', 'route3', 'ascent3', 'success3',
	 'route4', 'ascent4', 'success4'], axis=1, inplace=True)

### Termination Reason

In [328]:
df.termreason = apply_map(df.termreason, exped_termination_map)

### O2 Use

In [329]:
# check for inconsistent o2 use flags
df.loc[df.o2used == df.o2none].o2used.value_counts()

o2used
False    128
Name: count, dtype: int64

In [330]:
df.loc[
	df.o2unkwn,
	['o2used', 'o2none', 'o2climb', 'o2descent', 'o2medical', 'o2sleep', 'o2taken']
].sum()

o2used        0
o2none       18
o2climb       0
o2descent     0
o2medical     0
o2sleep       0
o2taken       0
dtype: int64

### Cleanup

In [331]:
df.loc[(df.bcdate.isna() | df.smtdate.isna()) & df.smtdays.notna()]

,expid,peakid,year,season,leaders,sponsor,claimed,disputed,approach,bcdate,...,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum,host_id
4,ANN279301_1979,ANN2,1979,3,Paul Moores,None,False,False,Pokhara->Marshyangdi->Pisang->Sabje Khola,None,...,None,None,None,None,False,False,None,None,2449204,0
22,ANN452301_1952,ANN4,1952,3,Kinji Imanishi,Japanese Alpine Club (JAC),False,False,Marshyangdi->Hongde->Sabje Khola,None,...,None,None,None,None,False,False,False,None,5012,0
23,ANN453101_1953,ANN4,1953,1,Basil R. Goodfellow,None,False,False,Seti Khola->Madi Khola,None,...,None,None,None,None,False,False,None,None,5013,0
24,ANN453301_1953,ANN4,1953,3,Toshio Imanishi,Academic Alpine Club of the University of Kyoto,False,False,Marshyangdi->Hongde->Sabje Khola,1953-10-21,...,None,None,None,None,False,False,None,None,5013,0
28,ANN469301_1969,ANN4,1969,3,Vladimir Prochazka,"Northern Bohemian Mountaineering Club, Libecec...",False,False,Marshyangdi->Hongde->Sabje Khola,None,...,None,None,None,None,False,False,None,None,2445531,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11567,TAKL24301_2024,TAKL,2024,3,Spencer Gray,Magical Nepal Takla Khang Expedition 2024,False,False,None,2024-10-04,...,None,Magical Nepal,False,False,False,False,False,None,5309,0
11568,TAKN24301_2024,TAKN,2024,3,Matthew Powell,Firante Takphu North Expedition 2024,False,False,None,None,...,None,Firante Treks,False,False,False,False,False,None,5309,0
11571,TUKU24301_2024,TUKU,2024,3,Giulia Monego,Italian-French Tukuche Peak Expedition 2024,False,False,None,2024-10-02,...,None,Sherpalaya,False,False,False,False,False,None,5309,0
11574,YASA04302_2004,YASA,2004,3,Maxime Belleville,French Yasa Thak Expedition 2004,False,False,None,None,...,None,None,False,False,False,False,False,None,3804,0


In [332]:
# recompute basecamp date where unavailable
df.loc[
	df.bcdate.isna() & df.smtdate.notna() & df.smtdays.notna(), 'bcdate'] = df.apply(
	lambda row: row['smtdate'] - pd.Timedelta(days=row['smtdays']), axis=1)

In [333]:
# recompute summit date where unavailable
df.loc[
	df.bcdate.notna() & df.smtdate.isna() & df.smtdays.notna(), 'smtdate'] = df.apply(
	lambda row: row['bcdate'] + pd.Timedelta(days=row['smtdays']), axis=1)

In [334]:
df.drop(
	['leaders', 'totmembers', 'smtmembers', 'mdeaths', 'tothired', 'smthired', 'hdeaths', 'nohired', 'smtdays', 'totdays'],
	axis=1, inplace=True)

In [335]:
df.head()

,expid,peakid,year,season,sponsor,claimed,disputed,approach,bcdate,smtdate,...,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum,host_id
0,ANN260101_1960,ANN2,1960,1,None,False,False,Marshyangdi->Hongde->Sabje Khola,1960-03-15,1960-05-17,...,None,None,None,None,False,False,None,None,2442047,0
1,ANN269301_1969,ANN2,1969,3,Mountaineering Club of Slovenia,False,False,Marshyangdi->Hongde->Sabje Khola,1969-09-25,1969-10-22,...,None,None,None,None,False,False,None,None,2445501,0
2,ANN273101_1973,ANN2,1973,1,Sangaku Doshikai Annapurna II Expedition 1973,False,False,Marshyangdi->Pisang->Salatang Khola,1973-03-16,1973-05-06,...,None,None,None,None,False,False,None,None,2446797,0
3,ANN278301_1978,ANN2,1978,3,British Annapurna II Expedition,False,False,Marshyangdi->Pisang->Salatang Khola,1978-09-08,1978-10-02,...,None,None,None,None,False,False,None,None,2448822,0
4,ANN279301_1979,ANN2,1979,3,None,False,False,Pokhara->Marshyangdi->Pisang->Sabje Khola,1979-10-18,1979-10-18,...,None,None,None,None,False,False,None,None,2449204,0
